<a href="https://colab.research.google.com/github/morrowbord/Spark/blob/main/%D0%A3%D1%80%D0%BE%D0%BA_2_3_%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D0%BD%D0%B0_%D1%83%D1%80%D0%BE%D0%BA%D0%B5_27dec2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Узнаем кол-во потоков

In [ ]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 53.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=f5be70708be892688ae7087f12744acddc726edd5e84284f75b8b34db111f996
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!./ngrok authtoken ВАШТОКЕН
get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

# Настройка SparkSession
Основные параметры:

 * -num-executors (spark.executor.instances) — количество экзекьютеров, 
 * --executor-memory (spark.executor.memory) — количество памяти у экзекьютера,
 * --execuor-cores (park.executor.cores) — количество ядер у экзекьютера.

На каждом сервере исполняется ОС, Hadoop/Yarn/OS Deamons, HDFS. Часть ресурсов потребляют они. HDFS (и другие файловые системы) имеют свою пропускную способность. Так же нужно учитывать сборщик мусора.

Есть две крайности: 1 ядро - 1 таск, 1 нода - 1 таск. Чаще всего оба варианта не самые удачные.

Рекомендация: не больше 5 ядер на 1 экзекьютор из-за пропуской способности HDFS.

## executor-memory
![Image of Yaktocat](https://miro.medium.com/max/1400/1*8yviZQXq9rXgBoup8W-kHA.png)

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.sql import SparkSession


spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 4)\
        .config('spark.executor.memory', f'{int(3000/1.1)}mb')\
        .config('spark.executor.cores', 1)\
        .getOrCreate()

sc = spark.sparkContext

In [ ]:
spark

# Пример из презентации

In [ ]:
rdd = sc.parallelize(['After all is', 'said and', 'done more is', 'said than done'])
print(f'Кол-во партиций: {rdd.getNumPartitions()}')

Кол-во партиций: 4


In [ ]:
# Как данные раскиданы по партициям
rdd.glom().collect()

[['After all is'], ['said and'], ['done more is'], ['said than done']]

In [ ]:
# Кол-во партиций можно изменить
rdd.repartition(2).glom().collect(), rdd.repartition(2).getNumPartitions()

([['After all is', 'done more is', 'said than done'], ['said and']], 2)

In [ ]:
rdd.coalesce(2).glom().collect()

[['After all is', 'said and'], ['done more is', 'said than done']]

In [ ]:
rdd.flatMap(lambda row: row.split())\
.map(lambda word: (word, 1))\
.reduceByKey(lambda a, b: a + b)\
.sortByKey(lambda row: row[0]).collect()

[('After', 1),
 ('all', 1),
 ('and', 1),
 ('done', 2),
 ('is', 2),
 ('more', 1),
 ('said', 2),
 ('than', 1)]

In [ ]:
# Создаём нужный нам rdd
rdd = rdd.flatMap(lambda row: row.split()).map(lambda row: (row, 1)).reduceByKey(lambda x, y: x + y)

In [ ]:
# Забираем данные (в  этот момент запускаются вычисления)
rdd.collect()

[('After', 1),
 ('and', 1),
 ('is', 2),
 ('said', 2),
 ('more', 1),
 ('than', 1),
 ('all', 1),
 ('done', 2)]

# Примеры RDD

 * In-Memory Processing
 * Immutability
 * Fault Tolerance
 * Lazy Evolution
 * Partitioning
 * Parallelize

## Считывание данных
Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification
Скачиваем и копируем в папку с нотебуком

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

In [ ]:
# Вспомогательная функция

def beautiful_print(rdd, num, title):
    print(title)
    for row in rdd.take(num):
        print('\t', row)

In [ ]:
data = sc.textFile('train.csv')

In [ ]:
data.take(3)

['battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range',
 '842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1',
 '1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2']

In [ ]:
data_header = data.first()
data_header

'battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range'

In [ ]:
data = data\
    .filter(lambda row: row != data_header)\
    .map(lambda row: [float(el) for el in row.split(',')])

In [ ]:
data.collect()

In [ ]:
beautiful_print(data, 30, 'Our rdd')

Our rdd
	 [842.0, 0.0, 2.2, 0.0, 1.0, 0.0, 7.0, 0.6, 188.0, 2.0, 2.0, 20.0, 756.0, 2549.0, 9.0, 7.0, 19.0, 0.0, 0.0, 1.0, 1.0]
	 [1021.0, 1.0, 0.5, 1.0, 0.0, 1.0, 53.0, 0.7, 136.0, 3.0, 6.0, 905.0, 1988.0, 2631.0, 17.0, 3.0, 7.0, 1.0, 1.0, 0.0, 2.0]
	 [563.0, 1.0, 0.5, 1.0, 2.0, 1.0, 41.0, 0.9, 145.0, 5.0, 6.0, 1263.0, 1716.0, 2603.0, 11.0, 2.0, 9.0, 1.0, 1.0, 0.0, 2.0]
	 [615.0, 1.0, 2.5, 0.0, 0.0, 0.0, 10.0, 0.8, 131.0, 6.0, 9.0, 1216.0, 1786.0, 2769.0, 16.0, 8.0, 11.0, 1.0, 0.0, 0.0, 2.0]
	 [1821.0, 1.0, 1.2, 0.0, 13.0, 1.0, 44.0, 0.6, 141.0, 2.0, 14.0, 1208.0, 1212.0, 1411.0, 8.0, 2.0, 15.0, 1.0, 1.0, 0.0, 1.0]
	 [1859.0, 0.0, 0.5, 1.0, 3.0, 0.0, 22.0, 0.7, 164.0, 1.0, 7.0, 1004.0, 1654.0, 1067.0, 17.0, 1.0, 10.0, 1.0, 0.0, 0.0, 1.0]
	 [1821.0, 0.0, 1.7, 0.0, 4.0, 1.0, 10.0, 0.8, 139.0, 8.0, 10.0, 381.0, 1018.0, 3220.0, 13.0, 8.0, 18.0, 1.0, 0.0, 1.0, 3.0]
	 [1954.0, 0.0, 0.5, 1.0, 0.0, 0.0, 24.0, 0.8, 187.0, 4.0, 0.0, 512.0, 1149.0, 700.0, 16.0, 3.0, 5.0, 1.0, 1.0, 1.0, 0.0]
	 [14

In [ ]:
col_num = data_header.split(',').index('n_cores')
data.map(lambda row: (row[col_num], 1)).reduceByKey(lambda x, y: x + y ).sortByKey().collect()

[(1.0, 242),
 (2.0, 247),
 (3.0, 246),
 (4.0, 274),
 (5.0, 246),
 (6.0, 230),
 (7.0, 259),
 (8.0, 256)]

In [ ]:
sorted(data.map(lambda row: (row[col_num], 1)).reduceByKey(lambda x, y: x + y ).collect())

PythonRDD[27] at RDD at PythonRDD.scala:53

In [ ]:
data.map(lambda row: (row[col_num], 1)).countByKey()

defaultdict(int,
            {1.0: 242,
             2.0: 247,
             3.0: 246,
             4.0: 274,
             5.0: 246,
             6.0: 230,
             7.0: 259,
             8.0: 256})

# DataFrame

In [ ]:
df = spark.read.csv('train.csv', header=True, inferSchema=True)

In [ ]:
schema = df.schema
schema

StructType(List(StructField(battery_power,IntegerType,true),StructField(blue,IntegerType,true),StructField(clock_speed,DoubleType,true),StructField(dual_sim,IntegerType,true),StructField(fc,IntegerType,true),StructField(four_g,IntegerType,true),StructField(int_memory,IntegerType,true),StructField(m_dep,DoubleType,true),StructField(mobile_wt,IntegerType,true),StructField(n_cores,IntegerType,true),StructField(pc,IntegerType,true),StructField(px_height,IntegerType,true),StructField(px_width,IntegerType,true),StructField(ram,IntegerType,true),StructField(sc_h,IntegerType,true),StructField(sc_w,IntegerType,true),StructField(talk_time,IntegerType,true),StructField(three_g,IntegerType,true),StructField(touch_screen,IntegerType,true),StructField(wifi,IntegerType,true),StructField(price_range,IntegerType,true)))

In [ ]:
df.take(10)

[Row(battery_power=842, blue=0, clock_speed=2.2, dual_sim=0, fc=1, four_g=0, int_memory=7, m_dep=0.6, mobile_wt=188, n_cores=2, pc=2, px_height=20, px_width=756, ram=2549, sc_h=9, sc_w=7, talk_time=19, three_g=0, touch_screen=0, wifi=1, price_range=1),
 Row(battery_power=1021, blue=1, clock_speed=0.5, dual_sim=1, fc=0, four_g=1, int_memory=53, m_dep=0.7, mobile_wt=136, n_cores=3, pc=6, px_height=905, px_width=1988, ram=2631, sc_h=17, sc_w=3, talk_time=7, three_g=1, touch_screen=1, wifi=0, price_range=2),
 Row(battery_power=563, blue=1, clock_speed=0.5, dual_sim=1, fc=2, four_g=1, int_memory=41, m_dep=0.9, mobile_wt=145, n_cores=5, pc=6, px_height=1263, px_width=1716, ram=2603, sc_h=11, sc_w=2, talk_time=9, three_g=1, touch_screen=1, wifi=0, price_range=2),
 Row(battery_power=615, blue=1, clock_speed=2.5, dual_sim=0, fc=0, four_g=0, int_memory=10, m_dep=0.8, mobile_wt=131, n_cores=6, pc=9, px_height=1216, px_width=1786, ram=2769, sc_h=16, sc_w=8, talk_time=11, three_g=1, touch_screen=0,

In [ ]:
df.show(1, vertical=True)

-RECORD 0-------------
 battery_power | 842  
 blue          | 0    
 clock_speed   | 2.2  
 dual_sim      | 0    
 fc            | 1    
 four_g        | 0    
 int_memory    | 7    
 m_dep         | 0.6  
 mobile_wt     | 188  
 n_cores       | 2    
 pc            | 2    
 px_height     | 20   
 px_width      | 756  
 ram           | 2549 
 sc_h          | 9    
 sc_w          | 7    
 talk_time     | 19   
 three_g       | 0    
 touch_screen  | 0    
 wifi          | 1    
 price_range   | 1    
only showing top 1 row



In [ ]:
df.dtypes

[('battery_power', 'int'),
 ('blue', 'int'),
 ('clock_speed', 'double'),
 ('dual_sim', 'int'),
 ('fc', 'int'),
 ('four_g', 'int'),
 ('int_memory', 'int'),
 ('m_dep', 'double'),
 ('mobile_wt', 'int'),
 ('n_cores', 'int'),
 ('pc', 'int'),
 ('px_height', 'int'),
 ('px_width', 'int'),
 ('ram', 'int'),
 ('sc_h', 'int'),
 ('sc_w', 'int'),
 ('talk_time', 'int'),
 ('three_g', 'int'),
 ('touch_screen', 'int'),
 ('wifi', 'int'),
 ('price_range', 'int')]

In [ ]:
df.printSchema()

root
 |-- battery_power: integer (nullable = true)
 |-- blue: integer (nullable = true)
 |-- clock_speed: double (nullable = true)
 |-- dual_sim: integer (nullable = true)
 |-- fc: integer (nullable = true)
 |-- four_g: integer (nullable = true)
 |-- int_memory: integer (nullable = true)
 |-- m_dep: double (nullable = true)
 |-- mobile_wt: integer (nullable = true)
 |-- n_cores: integer (nullable = true)
 |-- pc: integer (nullable = true)
 |-- px_height: integer (nullable = true)
 |-- px_width: integer (nullable = true)
 |-- ram: integer (nullable = true)
 |-- sc_h: integer (nullable = true)
 |-- sc_w: integer (nullable = true)
 |-- talk_time: integer (nullable = true)
 |-- three_g: integer (nullable = true)
 |-- touch_screen: integer (nullable = true)
 |-- wifi: integer (nullable = true)
 |-- price_range: integer (nullable = true)



In [ ]:
df.show(1, vertical=True)

-RECORD 0-------------
 battery_power | 842  
 blue          | 0    
 clock_speed   | 2.2  
 dual_sim      | 0    
 fc            | 1    
 four_g        | 0    
 int_memory    | 7    
 m_dep         | 0.6  
 mobile_wt     | 188  
 n_cores       | 2    
 pc            | 2    
 px_height     | 20   
 px_width      | 756  
 ram           | 2549 
 sc_h          | 9    
 sc_w          | 7    
 talk_time     | 19   
 three_g       | 0    
 touch_screen  | 0    
 wifi          | 1    
 price_range   | 1    
only showing top 1 row



In [ ]:
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|    battery_power|              blue|       clock_speed|          dual_sim|               fc|             four_g|        int_memory|             m_dep|        mobile_wt|           n_cores|               pc|         px_height|         px_width|               ram|             sc_h|             sc_w|        talk_time|          three_g|      touch_screen|              wifi|       price_range|
+-------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+------------------+-

In [ ]:
import pyspark.sql.functions as F

In [ ]:
'blue' * 2

'blueblue'

In [ ]:
F.col('blue')

Column<'blue'>

In [ ]:
df.select(F.col('blue') * 2, df['blue'].alias('a'), df.blue, 'blue').show()

+----------+---+----+----+
|(blue * 2)|  a|blue|blue|
+----------+---+----+----+
|         0|  0|   0|   0|
|         2|  1|   1|   1|
|         2|  1|   1|   1|
|         2|  1|   1|   1|
|         2|  1|   1|   1|
|         0|  0|   0|   0|
|         0|  0|   0|   0|
|         0|  0|   0|   0|
|         2|  1|   1|   1|
|         2|  1|   1|   1|
|         2|  1|   1|   1|
|         2|  1|   1|   1|
|         0|  0|   0|   0|
|         2|  1|   1|   1|
|         0|  0|   0|   0|
|         0|  0|   0|   0|
|         0|  0|   0|   0|
|         0|  0|   0|   0|
|         2|  1|   1|   1|
|         2|  1|   1|   1|
+----------+---+----+----+
only showing top 20 rows



In [ ]:
df.select(F.sum('blue'), ).show()

+---------+
|sum(blue)|
+---------+
|      990|
+---------+



Забыл рассказать про alias для переименовывния колонок, очень нужная функция

In [ ]:
df.select('blue', F.col('battery_power').alias('c'), df.blue * 2, df['blue'].alias('d'), F.col('blue') * 4)\
.withColumn('new_col', F.col('blue') * 5).show()

+----+----+----------+---+----------+-------+
|blue|   c|(blue * 2)|  d|(blue * 4)|new_col|
+----+----+----------+---+----------+-------+
|   0| 842|         0|  0|         0|      0|
|   1|1021|         2|  1|         4|      5|
|   1| 563|         2|  1|         4|      5|
|   1| 615|         2|  1|         4|      5|
|   1|1821|         2|  1|         4|      5|
|   0|1859|         0|  0|         0|      0|
|   0|1821|         0|  0|         0|      0|
|   0|1954|         0|  0|         0|      0|
|   1|1445|         2|  1|         4|      5|
|   1| 509|         2|  1|         4|      5|
|   1| 769|         2|  1|         4|      5|
|   1|1520|         2|  1|         4|      5|
|   0|1815|         0|  0|         0|      0|
|   1| 803|         2|  1|         4|      5|
|   0|1866|         0|  0|         0|      0|
|   0| 775|         0|  0|         0|      0|
|   0| 838|         0|  0|         0|      0|
|   0| 595|         0|  0|         0|      0|
|   1|1131|         2|  1|        

Стандартный групбай

In [ ]:
df.select('n_cores').groupby('n_cores').count().sort('n_cores').show()

+-------+-----+
|n_cores|count|
+-------+-----+
|      1|  242|
|      2|  247|
|      3|  246|
|      4|  274|
|      5|  246|
|      6|  230|
|      7|  259|
|      8|  256|
+-------+-----+



In [ ]:
df.select('*').groupby('n_cores').agg(F.count('*'), F.sum('n_cores')).show()

+-------+--------+------------+
|n_cores|count(1)|sum(n_cores)|
+-------+--------+------------+
|      1|     242|         242|
|      6|     230|        1380|
|      3|     246|         738|
|      5|     246|        1230|
|      4|     274|        1096|
|      8|     256|        2048|
|      7|     259|        1813|
|      2|     247|         494|
+-------+--------+------------+



In [ ]:
df.select('n_cores').groupby('n_cores')\
.agg(F.count('*').alias('a'), F.sum('n_cores').alias('b')).sort(F.col('n_cores').desc()).show()


# count().sort('n_cores').show()

+-------+---+----+
|n_cores|  a|   b|
+-------+---+----+
|      8|256|2048|
|      7|259|1813|
|      6|230|1380|
|      5|246|1230|
|      4|274|1096|
|      3|246| 738|
|      2|247| 494|
|      1|242| 242|
+-------+---+----+



toPandas() и createDataFrame позволяет нам конвертировать из пандас датафрейм в спарк датафрейм и обратно.

In [ ]:
import pandas as pd


type(pd.DataFrame(['a']))

pandas.core.frame.DataFrame

In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
spark.createDataFrame(pd.DataFrame(['a']).cast(IntegerType())).toPandas()

,0
0,a


In [ ]:
.withColumn('name', F.sha1())
.select("*", F.col() * F.col() )

In [ ]:
df.registerTempTable('df')

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [ ]:
spark.sql(f'select n_cores, count(*) from  df group by n_cores').show()

+-------+--------+
|n_cores|count(1)|
+-------+--------+
|      1|     242|
|      6|     230|
|      3|     246|
|      5|     246|
|      4|     274|
|      8|     256|
|      7|     259|
|      2|     247|
+-------+--------+

